In [1]:
# default_exp series.predict

# series.predict
> Methods for predicting MRI series types using a previously trained `RandomForestClassifier` trained with `scikit-learn`.

In [2]:
#export
from dicomtools.basics import *
from dicomtools.core import *
from dicomtools.preproc import *

from joblib import load

In [3]:
#export
path = Path('dicomtools/models')
_model_path = path/'mr-brain-series-select-rf.skl'

_y_names = pd.Index([
    't1', 
    't2',
    'swi', 
    'dwi', 
    'other', 
    'flair', 
    'adc', 
    'loc', 
    'spgr',
    'mra'
])

_features = ['MRAcquisitionType', 'AngioFlag', 'SliceThickness', 'RepetitionTime',
       'EchoTime', 'EchoTrainLength', 'PixelSpacing', 'ContrastBolusAgent',
       'InversionTime', 'DiffusionBValue', 'seq_E', 'seq_EP', 'seq_G',
       'seq_GR', 'seq_I', 'seq_IR', 'seq_M', 'seq_P', 'seq_R', 'seq_S',
       'seq_SE', 'var_E', 'var_K', 'var_MP', 'var_MTC', 'var_N', 'var_O',
       'var_OSP', 'var_P', 'var_S', 'var_SK', 'var_SP', 'var_SS', 'var_TOF',
       'opt_1', 'opt_2', 'opt_A', 'opt_ACC_GEMS', 'opt_B', 'opt_C', 'opt_D',
       'opt_E', 'opt_EDR_GEMS', 'opt_EPI_GEMS', 'opt_F', 'opt_FAST_GEMS',
       'opt_FC', 'opt_FC_FREQ_AX_GEMS', 'opt_FC_SLICE_AX_GEMS',
       'opt_FILTERED_GEMS', 'opt_FR_GEMS', 'opt_FS', 'opt_FSA_GEMS',
       'opt_FSI_GEMS', 'opt_FSL_GEMS', 'opt_FSP_GEMS', 'opt_FSS_GEMS', 'opt_G',
       'opt_I', 'opt_IFLOW_GEMS', 'opt_IR', 'opt_IR_GEMS', 'opt_L', 'opt_M',
       'opt_MP_GEMS', 'opt_MT', 'opt_MT_GEMS', 'opt_NPW', 'opt_P', 'opt_PFF',
       'opt_PFP', 'opt_PROP_GEMS', 'opt_R', 'opt_RAMP_IS_GEMS', 'opt_S',
       'opt_SAT1', 'opt_SAT2', 'opt_SAT_GEMS', 'opt_SEQ_GEMS', 'opt_SP',
       'opt_T', 'opt_T2FLAIR_GEMS', 'opt_TRF_GEMS', 'opt_VASCTOF_GEMS',
       'opt_VB_GEMS', 'opt_W', 'opt_X', 'opt__']

# need to add step to `preprocess()` to force these features as cols

In [4]:
#export
def _get_preds(clf, df, features, y_names=_y_names):
    y_pred = clf.predict(df[features])
    y_prob = clf.predict_proba(df[features])
    preds = pd.Series(y_names.take(y_pred))
    probas = pd.Series([y_prob[i][pred] for i, pred in enumerate(y_pred)])
    return pd.DataFrame({'seq_pred': preds, 'pred_proba': probas})


In [5]:
#export
def predict_from_df(df, features=_features, thresh=0.8, path=_model_path, clf=None):
    "Predict series from df[features] at confidence threshold `p >= thresh`"
    if 'plane' not in df.columns:
        df1 = preprocess(df)
        labels = extract_labels(df1)
        df1 = df1.join(labels[['plane', 'contrast', 'seq_label']])
    else:
        df1 = df.copy()
    if not clf:
        clf = load(path)
    df1 = df1.join(_get_preds(clf, df1, features))
    filt = df1['pred_proba'] < thresh
    df1['seq_pred'][filt] = 'unknown'
    return df1
